## DNN model for MNIST dataset

In [1]:
import numpy as np
import torch
import tensorflow as tf

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Convert to PyTorch format [batch_size, total pixels]
# Since images are already normalized and flattened for TensorFlow, we can use the same arrays
train_images_pt = torch.tensor(train_images_tf).float()
test_images_pt = torch.tensor(test_images_tf).float()
train_labels_pt = torch.tensor(train_labels)
test_labels_pt = torch.tensor(test_labels)

ModuleNotFoundError: No module named 'tensorflow'

In [12]:
from tensorflow import keras
num_classes = 10

model_tf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28*28,)),  # Adjusted for 28x28 images
#    keras.layers.Dense(128, activation='relu'),     # Increased number of neurons
    keras.layers.Dense(56, activation='relu'),      # Additional hidden layer
    keras.layers.Dense(num_classes, activation='softmax')  # Output layer for 10 classes
])

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [13]:
# Train the model
model_tf.fit(train_images_tf, train_labels, epochs=3, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/3


2024-01-28 16:04:21.886799: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.2KiB (rounded to 2304)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-28 16:04:21.886822: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-01-28 16:04:21.886827: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 59, Chunks in use: 59. 14.8KiB allocated for chunks. 14.8KiB in use in bin. 3.2KiB client-requested in use in bin.
2024-01-28 16:04:21.886830: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-01-28 16:04:21.886832: I external/local_tsl/

ResourceExhaustedError: in user code:

    File "/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py", line 1154, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/optimizers/optimizer.py", line 544, in minimize
        self.apply_gradients(grads_

### Convert to Pytorch DNN model

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        # Fully connected layers / Dense block
        # First dense layer
        self.fc1 = nn.Linear(28*28, 56)  # Flatten 28*28 and feed into 56 neurons
        
        # Second dense layer (output layer)
        self.fc2 = nn.Linear(56, num_classes)  # 56 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Flatten the tensor
        #x = x.view(-1, 28*28)
        
        # Fully connected layers with ReLU activation for the first layer
        x = F.relu(self.fc1(x))

        # Output layer with no activation
        # Softmax will be applied externally during training and evaluation
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)
    
model_pt = Net()

In [16]:
# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[1].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

In [17]:
# Select the image for TensorFlow and PyTorch
controlled_input_tf = test_images[38].reshape(1, 28*28)  # Reshape to (1, 784) for DNN
controlled_input_pt = torch.from_numpy(controlled_input_tf).float()

# Test TensorFlow Model
output_tf = model_tf.predict(controlled_input_tf) 
print("TensorFlow Basic Model Output:", output_tf)

# Test PyTorch Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)
print("PyTorch Basic Model Output:", torch.exp(output_pt).numpy())

1/1 [==============================] - 0s 36ms/step
TensorFlow Basic Model Output: [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
PyTorch Basic Model Output: [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [18]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDataset for test data
test_dataset = TensorDataset(test_images_pt, test_labels_pt)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def evaluate_pytorch_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_loader)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')


Accuracy of the PyTorch model on the test images: 96.61000000%


In [19]:
def get_predictions_tf(model, test_images, batch_size=32):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

def get_predictions_pt(model, test_images, batch_size=32):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(0, len(test_images), batch_size):
            batch = test_images[i:i+batch_size]
            pred = model(batch)
            predictions.extend(torch.argmax(pred, axis=1).tolist())
    return predictions

In [10]:
# Generate predictions
predictions_tf = get_predictions_tf(model_tf, test_images_tf)
predictions_pt = get_predictions_pt(model_pt, test_images_pt)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images_tf)} samples")


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 20ms/step
Number of mismatches: 0 out of 10000 samples


In [14]:
# Resize and Normalize x_test_image from float64 to int8
x_test_image_norm = (test_images / 255.0 * 255 - 128).astype(np.int8)
x_train_image_norm = (train_images / 255.0 * 255 - 128).astype(np.int8)

In [15]:
import tensorflow as tf

# Create a converter
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)

# Indicate that you want to perform default optimizations,
# which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define a generator function that provides your test data's numpy arrays
def representative_data_gen():
  for i in range(500):
    yield [np.array(train_images[i:i+1], dtype=np.float32)]

# Use the generator function to guide the quantization process
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
tflite_model = converter.convert()

# Save the model to disk
open("q_aware_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpwhl7byf9/assets


INFO:tensorflow:Assets written to: /tmp/tmpwhl7byf9/assets
/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-01-28 12:40:59.650471: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-28 12:40:59.650491: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-28 12:40:59.650629: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpwhl7byf9
2024-01-28 12:40:59.651605: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-28 12:40:59.651614: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpwhl7byf9
2024-01-28 12:40:59.654288: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
20

46896

In [16]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="q_aware_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# The input needs to be quantized, so we retrieve the quantization parameters
input_scale, input_zero_point = input_details[0]['quantization']
output_scale, output_zero_point = output_details[0]['quantization']

# Normalize and quantize the test images
test_images_quant = (test_images / input_scale + input_zero_point).astype(np.int8)



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [17]:
# Evaluate the quantized TFLite model
correct_predictions = 0
for i in range(len(test_images)):
    test_image = np.expand_dims(test_images_quant[i], axis=0)
    
    # Set the value for the input tensor
    interpreter.set_tensor(input_details[0]['index'], test_image)
    
    # Run the inference
    interpreter.invoke()

    # Retrieve the output and dequantize
    output = interpreter.get_tensor(output_details[0]['index'])
    output = np.argmax(output, axis=1)
    predicted_class = output[0]
    if predicted_class == test_labels[i]:
        correct_predictions += 1

# Calculate the accuracy
accuracy = correct_predictions / len(test_images) * 100
print(f'Quantized model accuracy: {accuracy:.2f}%')

ValueError: Cannot set tensor: Dimension mismatch. Got 3 but expected 2 for input 0.

: 

### Test on Orion


In [5]:
!scarb new orion

Created `orion` package.


In [11]:
import numpy as np
import tensorflow as tf
import os


# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="q_aware_model.tflite")
interpreter.allocate_tensors()

# Create an object with all tensors (an input + all weights and biases)
tensors = {
    "fc1_weights": interpreter.get_tensor(1), 
    "fc1_bias": interpreter.get_tensor(2), 
    "fc2_weights": interpreter.get_tensor(4), 
    "fc2_bias": interpreter.get_tensor(5)
}

# Create the directory if it doesn't exist
os.makedirs('./orion_dnn/src/generated', exist_ok=True)


In [20]:
def to_fixed_point(val, bits):
    return round(val * (2**bits))

def mnist_image_to_fixed_point(data):
    return [to_fixed_point(val.item(), 16) for val in data]


def generate_input_cairo(data):
    values = mnist_image_to_fixed_point(data)
    values = [f"FixedTrait::<FP16x16>::new({val}, {'true' if val < 0 else 'false'})" for val in values]
    return ",\n ".join(values)

input_cairo = generate_input_cairo(controlled_input_tf[0])

with open("orion_dnn/src/generated/input.cairo", "w") as f:
    f.write("""
use array::{SpanTrait, ArrayTrait};
use orion::operators::tensor::{TensorTrait, FP16x16Tensor, Tensor};
use orion::numbers::{FixedTrait, FP16x16};
fn input() -> Tensor<FP16x16> {
    TensorTrait::<FP16x16>::new(
        array![196].span(),
        array![
    """)
    f.write(input_cairo)
    f.write("""
        ].span()
    )
}
    """)

In [12]:

for tensor_name, tensor in tensors.items():
    with open(os.path.join('./orion_dnn/src', 'generated', f"{tensor_name}.cairo"), "w") as f:
        f.write(
            "use array::{ArrayTrait, SpanTrait};\n" +
            "use orion::operators::tensor::{core::{Tensor, TensorTrait}};\n" +
            "use orion::operators::tensor::FP16x16Tensor;\n" +
            "use orion::numbers::fixed_point::implementations::fp16x16::core::{FP16x16, FixedTrait};\n" +
            "\n" + f"fn {tensor_name}() -> Tensor<FP16x16>" + "{\n\n" + 
            "let mut shape = ArrayTrait::new();\n"
        )
        for dim in tensor.shape:
            f.write(f"shape.append({dim});\n")
        f.write("let mut data = ArrayTrait::new();\n")

        for val in np.nditer(tensor.flatten()):
            f.write("    data.append(i32 {{ mag: {0}, sign: {1} }});\n".format(abs(int(val)), str(val < 0).lower()))
        f.write(
            "    TensorTrait::new(shape.span(), data.span())\n" +
            "}\n"
        )

with open(os.path.join('./orion_dnn/src', 'generated.cairo'), 'w') as f:
    for param_name in tensors.keys():
        f.write(f"mod {param_name};\n")

In [11]:
import tensorflow_model_optimization as tfmot

# Apply quantization to the layers
quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model = quantize_model(model_tf)

# 'quantize_model' requires a recompile
q_aware_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

q_aware_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 784)               3         
 yer)                                                            
                                                                 
 quant_dense (QuantizeWrapp  (None, 56)                43965     
 erV2)                                                           
                                                                 
 quant_dense_1 (QuantizeWra  (None, 10)                575       
 pperV2)                                                         
                                                                 
Total params: 44543 (174.00 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 13 (52.00 Byte)
_________________________________________________________________


In [12]:
batch_size = 256
epochs = 3
history = q_aware_model.fit(train_images_tf, train_labels,
                            epochs=epochs,
                            validation_split=0.2)



Epoch 1/3


2024-01-28 12:38:59.657971: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 12:38:59.658006: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 12:38:59.659413: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 12:38:59.659424: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 12:38:59.659437: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28

1500/1500 [==============================] - 3s 2ms/step - loss: 0.0963 - accuracy: 0.9720 - val_loss: 0.1078 - val_accuracy: 0.9672
Epoch 2/3
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0788 - accuracy: 0.9758 - val_loss: 0.0958 - val_accuracy: 0.9718
Epoch 3/3
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0657 - accuracy: 0.9806 - val_loss: 0.0903 - val_accuracy: 0.9733


In [13]:
import tensorflow as tf

# Create a converter
converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)

# Indicate that you want to perform default optimizations,
# which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define a generator function that provides your test data's numpy arrays
def representative_data_gen():
  for i in range(500):
    yield [test_images_tf[i:i+1]]

# Use the generator function to guide the quantization process
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
tflite_model = converter.convert()

# Save the model to disk
open("q_aware_model.tflite", "wb").write(tflite_model)



INFO:tensorflow:Assets written to: /tmp/tmp4j4dhfnv/assets


INFO:tensorflow:Assets written to: /tmp/tmp4j4dhfnv/assets
/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-01-28 12:39:13.507507: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-28 12:39:13.507524: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-28 12:39:13.507747: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4j4dhfnv
2024-01-28 12:39:13.508200: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-28 12:39:13.508209: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4j4dhfnv
2024-01-28 12:39:13.509435: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 op

ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_input_1:0 

In [9]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="q_aware_model.tflite")
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [29]:
# Select the image for TensorFlow and PyTorch
controlled_input_tf = test_images[34].reshape(1, 28*28)  # Reshape to (1, 784) for DNN
controlled_input_pt = torch.from_numpy(controlled_input_tf).float()

In [30]:
# Create an object with all tensors 
#(an input + all weights and biases)
tensors = {
    "input": controlled_input_pt,
    "fc1_weights": interpreter.get_tensor(1), 
    "fc1_bias": interpreter.get_tensor(2), 
    "fc2_weights": interpreter.get_tensor(4), 
    "fc2_bias": interpreter.get_tensor(5)
}

In [31]:
import os
# Create the directory if it doesn't exist
os.makedirs('./orion_dnn/src/generated', exist_ok=True)

for tensor_name, tensor in tensors.items():
    with open(os.path.join('./orion_dnn/src', 'generated', f"{tensor_name}.cairo"), "w") as f:
        f.write(
            "use core::array::ArrayTrait;\n" +
            "use orion::operators::tensor::{TensorTrait, Tensor, I32Tensor};\n" +
            "use orion::numbers::i32;\n\n" +
            "\nfn {0}() -> Tensor<i32> ".format(tensor_name) + "{\n" +
            "    let mut shape = ArrayTrait::<usize>::new();\n"
        )
        for dim in tensor.shape:
            f.write("    shape.append({0});\n".format(dim))
        f.write(
            "    let mut data = ArrayTrait::<i32>::new();\n"
        )
        for val in np.nditer(tensor.flatten()):
            f.write("    data.append(i32 {{ mag: {0}, sign: {1} }});\n".format(abs(int(val)), str(val < 0).lower()))
        f.write(
            "    TensorTrait::new(shape.span(), data.span())\n" +
            "}\n"
        )
      
with open(os.path.join('./orion_dnn/src', 'generated.cairo'), 'w') as f:
    for param_name in tensors.keys():
        f.write(f"mod {param_name};\n")


In [32]:
! touch orion_dnn/src/nn.cairo


In [33]:
%%writefile orion_dnn/src/nn.cairo
use orion::operators::tensor::core::Tensor;
use orion::numbers::signed_integer::{integer_trait::IntegerTrait, i32::i32};
use orion::operators::nn::{NNTrait, I32NN};

fn fc1(i: Tensor<i32>, w: Tensor<i32>, b: Tensor<i32>) -> Tensor<i32> {
    let x = NNTrait::linear(i, w, b);
    NNTrait::relu(@x)
}

fn fc2(i: Tensor<i32>, w: Tensor<i32>, b: Tensor<i32>) -> Tensor<i32> {
    NNTrait::linear(i, w, b)
}


Overwriting orion_dnn/src/nn.cairo


In [34]:
! touch orion_dnn/src/test.cairo

In [35]:
%%writefile orion_dnn/src/test.cairo
use core::array::SpanTrait;

use mnist_nn::nn::fc1;
use mnist_nn::nn::fc2;
use mnist_nn::generated::input::input;
use mnist_nn::generated::fc1_bias::fc1_bias;
use mnist_nn::generated::fc1_weights::fc1_weights;
use mnist_nn::generated::fc2_bias::fc2_bias;
use mnist_nn::generated::fc2_weights::fc2_weights;

use orion::operators::tensor::I32Tensor;

#[test]
#[available_gas(99999999999999999)]
fn mnist_nn_test() {
    let input = input();
    let fc1_bias = fc1_bias();
    let fc1_weights = fc1_weights();
    let fc2_bias = fc2_bias();
    let fc2_weights = fc2_weights();

    let x = fc1(input, fc1_weights, fc1_bias);
    let x = fc2(x, fc2_weights, fc2_bias);

    let x = *x.argmax(0, Option::None(()), Option::None(())).data.at(0);

    assert(x == 3, 'should predict 2');
}



Overwriting orion_dnn/src/test.cairo


In [36]:
! cd orion_dnn & scarb run test

error: failed to read manifest at: /home/guy1m0/Desktop/ZKML-Benchmark/Milestone 2/MNIST-DNN/Scarb.toml

Caused by:
    No such file or directory (os error 2)


Use Giza-cli to automatically generate cairo script

In [46]:
def to_fixed_point(val, bits):
    return round(val * (2**bits))

def mnist_image_to_fixed_point(data):
    return [to_fixed_point(val.item(), 16) for val in data]


def generate_input_cairo(data):
    values = mnist_image_to_fixed_point(data)
    values = [f"FixedTrait::<FP16x16>::new({val}, {'true' if val < 0 else 'false'})" for val in values]
    return ",\n ".join(values)

In [47]:
controlled_input_pt.shape

torch.Size([1, 784])

In [48]:
input_cairo = generate_input_cairo(controlled_input_pt[0])

In [51]:
with open("mnist_cairo/src/input.cairo", "w") as f:
    f.write("""
use array::{SpanTrait, ArrayTrait};
use orion::operators::tensor::{TensorTrait, FP16x16Tensor, Tensor};
use orion::numbers::{FixedTrait, FP16x16};
fn input() -> Tensor<FP16x16> {
    TensorTrait::<FP16x16>::new(
        array![196].span(),
        array![
    """)
    f.write(input_cairo)
    f.write("""
        ].span()
    )
}
    """)